In [1]:
# @title Setup

competition = "crime-charges-analysis"  # @param
# @markdown ---

from google.colab import userdata
import json

# Get the Kaggle credentials from Colab's userdata
username = userdata.get("KAGGLE_USER")
key = userdata.get("KAGGLE_KEY")

# Echo the credentials into the kaggle.json file
!mkdir -p ~/.kaggle
!echo '{{"username":"{username}","key":"{key}"}}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

competition_name = "crime-charges-analysis"
dir_name = "Crime-charges"

! kaggle competitions download -c {competition_name}
! mkdir {dir_name}
! unzip /content/{competition_name}.zip -d {dir_name}

  0% 0.00/215k [00:00<?, ?B/s]
100% 215k/215k [00:00<00:00, 535MB/s]
Archive:  /content/crime-charges-analysis.zip
  inflating: Crime-charges/classes.xlsx  
  inflating: Crime-charges/submission.csv  
  inflating: Crime-charges/train.csv  


In [2]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [3]:
import pandas as pd
import numpy as np

from unsloth import FastLanguageModel
from datasets import Dataset
from tqdm import tqdm
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [4]:
df = pd.read_csv('/content/Crime-charges/train.csv')
df.head()

,id,story,answers
0,OhbVrpoiVg,จุงลิ่วขโมยรถจักรยานยนต์จากลานจอด ขณะที่แคสใช้...,"[{'จุงลิ่ว': '[2]'}, {'แคส': '[2,5]'}]"
1,RV5IfLBcbf,จิรศักก์กำลังทำอาหารอยู่ในครัว ในขณะที่ไขนภาเด...,"[{'จิรศักก์': '[1]'}, {'ไขนภา': '[1]'}]"
2,noGMbJmTPS,คมกาญจน์ขโมยเงินสดจากตู้อัตโนมัติในห้างและบุกร...,"[{'คมกาญจน์': '[2,4,5]'}, {'จิรานุวัตร': '[6]'}]"
3,IAoCLrZ3aW,ขจรพงค์ปล่อยให้ระบบลิฟต์โรงแรมขัดข้องจนตกลงมาพ...,[{'ขจรพงค์': '[3]'}]
4,ZkSBvrjn9W,กิมใจยิงเจ้าหน้าที่ขนส่งเสียชีวิตขณะนำของเข้าร...,"[{'กิมใจ': '[2,5,7]'}, {'ขนิฐชีพ': '[5]'}]"


## Unsloth

In [5]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-14B",
    max_seq_length = 1024,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
    dtype=None,
)

==((====))==  Unsloth 2025.5.4: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.4 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


**Prep data format**

In [7]:
classes = pd.read_excel('/content/Crime-charges/classes.xlsx')

description = ""
for _, row in classes.iterrows():
    description += f"{row['Class']}: {row['Description']}\n"

# print(description)

In [8]:
story_set = []
for i, row in tqdm(df.iterrows()):
    story_set.append({'conversations': [
        {'content': f"""## Description:
You are AI that read storyline and separate the actions of each person.

 This is the class description:
{description}

#### กฎพิเศษ:
- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง
- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]
- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**
- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**
- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**
- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**

- การเสียชีวิต:
- หากเกิดจากความตั้งใจ → [5, 7]
- หากเกิดจากความประมาท → [3, 7]
- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “เสียชีวิต” → ห้ามใส่ [7]
- ฆ่าตัวตาย → ไม่ถือว่าเป็นความผิด
- ตายเพราะโรค → ไม่นับเป็นความผิด

- หากมีการ “ลักทรัพย์”, “ขโมย”, หรือ “ชิงทรัพย์”:
- แล้วมีการ “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” → ใส่ **[2, 5]**
- ห้ามใส่ [5] หากในข้อความไม่มีคำชัดเจน เช่น “ข่มขู่”, “ทำร้าย”, “ใช้อาวุธ”, “ทำให้เจ็บ”, “ทำให้เลือดออก”, “ยิง”, “แทง”, “ตบ”, “บีบคอ” หรือคำใกล้เคียง
- หากไม่มีการข่มขู่หรือทำร้ายร่างกาย → ใส่เฉพาะ **[2]**

- หากความผิดเกิดจาก “เมาแล้วขับ” หรือ “หลับใน” → ใส่เฉพาะ **[3]** และ **ห้ามใส่ [5]**

- หากมีการ "ฉ้อโกง", "ปลอมเอกสาร" หรือ "หลอกลวง":
- แล้วมีคำชัดเจนว่า “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” ปรากฏในข้อความ → ใส่ **[5, 6]**
- ห้ามใส่ [5] หากในข้อความไม่มีคำชัดเจน เช่น “ข่มขู่”, “ทำร้าย”, “ใช้อาวุธ”, “ทำให้เจ็บ”, “ทำให้เลือดออก”, “ยิง”, “แทง”, “ตบ”, “บีบคอ” หรือคำใกล้เคียง
- หากไม่มีการข่มขู่หรือทำร้ายร่างกาย → ใส่เฉพาะ **[6]**


- หากบุคคล:
- เพียงถูกกล่าวถึง, ไม่ได้กระทำสิ่งใดผิด
- เป็นผู้ถูกกระทำ หรือผู้เสียหาย  → ใส่ **[1]**

This is the incident:
{row['story']}
""",
         'role': 'user'},
        {'content': row['answers'], 'role': 'assistant'}
    ]})

story_set[0]['conversations']

1800it [00:00, 20641.71it/s]


[{'content': '## Description:\nYou are AI that read storyline and separate the actions of each person.\n\n This is the class description:\n1: ไม่มีความผิดหรือไม่มีความเกี่ยวข้องกับเหตุการณ์\n2: ลักทรัพย์หรือชิงทรัพย์\n3: กระทำการใดๆด้วยความประมาทหรือละเลยส่งผลให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน\n4: บุกรุกเข้าเขตหวงห้ามหรือนอกเวลาทำการ\n5: กระทำการใดๆด้วยเจตนาที่ส่งผลให้เกิดความเสียหายทางด้านร่างกาย,จิตใจหรือทรัพย์สิน\n6: ฉ้อโกงประชาชน, ปลอมแปลงเอกสาร หรือปลอมแปลงสิ่งของต่างๆ\n7: ทำให้มีคนตายด้วยวิธีการใดๆก็ตาม\n\n\n#### กฎพิเศษ:\n- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง\n- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]\n- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**\n- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**\n- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**\n- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**\n\n- การเสียชีวิต:\n- หากเกิดจากความตั้งใจ → [5, 7]\n- หากเกิดจากความประมาท → [3, 7]\n- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “เสียช

**Dataset**

In [9]:
from unsloth.chat_templates import standardize_sharegpt

dataset = standardize_sharegpt(Dataset.from_list(story_set))
non_reasoning_conversations = tokenizer.apply_chat_template(
    dataset["conversations"],
    tokenize = False,
)

non_reasoning_conversations[0]

Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/1800 [00:00<?, ? examples/s]

'<|im_start|>user\n## Description:\nYou are AI that read storyline and separate the actions of each person.\n\n This is the class description:\n1: ไม่มีความผิดหรือไม่มีความเกี่ยวข้องกับเหตุการณ์\n2: ลักทรัพย์หรือชิงทรัพย์\n3: กระทำการใดๆด้วยความประมาทหรือละเลยส่งผลให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน\n4: บุกรุกเข้าเขตหวงห้ามหรือนอกเวลาทำการ\n5: กระทำการใดๆด้วยเจตนาที่ส่งผลให้เกิดความเสียหายทางด้านร่างกาย,จิตใจหรือทรัพย์สิน\n6: ฉ้อโกงประชาชน, ปลอมแปลงเอกสาร หรือปลอมแปลงสิ่งของต่างๆ\n7: ทำให้มีคนตายด้วยวิธีการใดๆก็ตาม\n\n\n#### กฎพิเศษ:\n- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง\n- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]\n- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**\n- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**\n- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**\n- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**\n\n- การเสียชีวิต:\n- หากเกิดจากความตั้งใจ → [5, 7]\n- หากเกิดจากความประมาท → [3, 7]\n- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “

In [10]:
data = pd.Series(non_reasoning_conversations)
data.name = "text"

combined_dataset = Dataset.from_pandas(pd.DataFrame(data))
combined_dataset = combined_dataset.shuffle(seed=3407)
combined_dataset[0]['text']

'<|im_start|>user\n## Description:\nYou are AI that read storyline and separate the actions of each person.\n\n This is the class description:\n1: ไม่มีความผิดหรือไม่มีความเกี่ยวข้องกับเหตุการณ์\n2: ลักทรัพย์หรือชิงทรัพย์\n3: กระทำการใดๆด้วยความประมาทหรือละเลยส่งผลให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน\n4: บุกรุกเข้าเขตหวงห้ามหรือนอกเวลาทำการ\n5: กระทำการใดๆด้วยเจตนาที่ส่งผลให้เกิดความเสียหายทางด้านร่างกาย,จิตใจหรือทรัพย์สิน\n6: ฉ้อโกงประชาชน, ปลอมแปลงเอกสาร หรือปลอมแปลงสิ่งของต่างๆ\n7: ทำให้มีคนตายด้วยวิธีการใดๆก็ตาม\n\n\n#### กฎพิเศษ:\n- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง\n- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]\n- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**\n- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**\n- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**\n- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**\n\n- การเสียชีวิต:\n- หากเกิดจากความตั้งใจ → [5, 7]\n- หากเกิดจากความประมาท → [3, 7]\n- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “

### Model Training

In [11]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = combined_dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 2, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 30,
        learning_rate = 2e-4, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=8):   0%|          | 0/1800 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,800 | Num Epochs = 1 | Total steps = 225
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 2 x 1) = 8
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.121900
2,1.121900
3,1.100400
4,1.022000
5,0.915800
6,0.800500
7,0.686200
8,0.584900
9,0.484100
10,0.378600


KeyboardInterrupt: 

In [12]:
from google.colab import userdata

if False:
    model.save_pretrained("Qwen3_14B-crime", tokenizer, save_method="lora",)
if False:
    model.push_to_hub_merged("PaweekornSora/Qwen3_14B-crime", tokenizer, save_method="lora",
                             token=userdata.get("HF_ADMIN"))

Unsloth: Saving LoRA adapters. Please wait...


README.md:   0%|          | 0.00/594 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/514M [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/PaweekornSora/Qwen3_14B-crime


## Inference

In [12]:
test_df = pd.read_csv('/content/Crime-charges/submission.csv')
test_df.head()

,id,story,answers
0,XwlpP5otmI,เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต,"[{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]"
1,1pUXZYyXqQ,จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์,[{'จิดาพัฒน์': '[1]'}]
2,drf7jObOL3,กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให...,"[{'กรัณยพร': '[3,7]'}]"
3,ghD7Mhgjsq,คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอม...,NaN
4,5lfeF2x61n,กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้...,NaN


In [13]:
test_set = []
for i, row in tqdm(test_df.iterrows()):
    test_set.append({'conversations': [
        {'content': f"""## Description:
You are AI that read storyline and separate the actions of each person.

 This is the class description:
{description}

#### กฎพิเศษ:
- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง
- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]
- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**
- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**
- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**
- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**

- การเสียชีวิต:
- หากเกิดจากความตั้งใจ → [5, 7]
- หากเกิดจากความประมาท → [3, 7]
- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “เสียชีวิต” → ห้ามใส่ [7]
- ฆ่าตัวตาย → ไม่ถือว่าเป็นความผิด
- ตายเพราะโรค → ไม่นับเป็นความผิด

- หากมีการ “ลักทรัพย์”, “ขโมย”, หรือ “ชิงทรัพย์”:
- แล้วมีการ “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” → ใส่ **[2, 5]**
- ห้ามใส่ [5] หากในข้อความไม่มีคำชัดเจน เช่น “ข่มขู่”, “ทำร้าย”, “ใช้อาวุธ”, “ทำให้เจ็บ”, “ทำให้เลือดออก”, “ยิง”, “แทง”, “ตบ”, “บีบคอ” หรือคำใกล้เคียง
- หากไม่มีการข่มขู่หรือทำร้ายร่างกาย → ใส่เฉพาะ **[2]**

- หากความผิดเกิดจาก “เมาแล้วขับ” หรือ “หลับใน” → ใส่เฉพาะ **[3]** และ **ห้ามใส่ [5]**

- หากมีการ "ฉ้อโกง", "ปลอมเอกสาร" หรือ "หลอกลวง":
- แล้วมีคำชัดเจนว่า “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” ปรากฏในข้อความ → ใส่ **[5, 6]**
- ห้ามใส่ [5] หากในข้อความไม่มีคำชัดเจน เช่น “ข่มขู่”, “ทำร้าย”, “ใช้อาวุธ”, “ทำให้เจ็บ”, “ทำให้เลือดออก”, “ยิง”, “แทง”, “ตบ”, “บีบคอ” หรือคำใกล้เคียง
- หากไม่มีการข่มขู่หรือทำร้ายร่างกาย → ใส่เฉพาะ **[6]**


- หากบุคคล:
- เพียงถูกกล่าวถึง, ไม่ได้กระทำสิ่งใดผิด
- เป็นผู้ถูกกระทำ หรือผู้เสียหาย  → ใส่ **[1]**

This is the incident:
{row['story']}
""", 'role': 'user'}]})

test_dataset = Dataset.from_list(test_set)
test_dataset = standardize_sharegpt(test_dataset)
test_dataset = tokenizer.apply_chat_template(
    test_dataset["conversations"],
    tokenize = False,
)
test_dataset[0]

1000it [00:00, 23800.98it/s]


Unsloth: Standardizing formats (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

'<|im_start|>user\n## Description:\nYou are AI that read storyline and separate the actions of each person.\n\n This is the class description:\n1: ไม่มีความผิดหรือไม่มีความเกี่ยวข้องกับเหตุการณ์\n2: ลักทรัพย์หรือชิงทรัพย์\n3: กระทำการใดๆด้วยความประมาทหรือละเลยส่งผลให้เกิดความเสียหายต่อชีวิตหรือทรัพย์สิน\n4: บุกรุกเข้าเขตหวงห้ามหรือนอกเวลาทำการ\n5: กระทำการใดๆด้วยเจตนาที่ส่งผลให้เกิดความเสียหายทางด้านร่างกาย,จิตใจหรือทรัพย์สิน\n6: ฉ้อโกงประชาชน, ปลอมแปลงเอกสาร หรือปลอมแปลงสิ่งของต่างๆ\n7: ทำให้มีคนตายด้วยวิธีการใดๆก็ตาม\n\n\n#### กฎพิเศษ:\n- แต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง\n- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]\n- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**\n- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**\n- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**\n- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**\n\n- การเสียชีวิต:\n- หากเกิดจากความตั้งใจ → [5, 7]\n- หากเกิดจากความประมาท → [3, 7]\n- หากไม่มีคำว่า “ฆ่า”, “ตาย”, “

In [15]:
from transformers import TextStreamer

result = []
for text in tqdm(test_dataset):
    answer = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 2048, # Increase for longer outputs!
        temperature = 0.2, top_p = 0.8, top_k = 20, # For non thinking
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )

    decoded = tokenizer.decode(answer[0], skip_special_tokens=True)
    result.append(decoded)

  0%|          | 0/1000 [00:00<?, ?it/s]

สิ่งใด
- เป็นผู้ถูกกระทำสิ่งใด, ไม่ใช่ผู้กระทำสิ่งใดผิด
- แล้วแต่ละบุคคลสามารถมีความผิดได้สูงสุดไม่เกิน **3 ข้อ** ต่อครั้ง
- หากมีหลายความผิด ให้แสดงหมายเลขเรียงจากน้อยไปมาก เช่น [2, 5] ไม่ใช่ [5, 2]
- หาก “ทำให้เสียชีวิตโดยเจตนา” → ใส่ **[5, 7]**
- หาก “ทำให้เสียชีวิตโดยประมาท” → ใส่ **[3, 7]**
- **[7] ต้องใช้ร่วมกับ [3] หรือ [5] เท่านั้น**
- **ห้ามใช้ [3] และ [5] พร้อมกันในกรณีเดียวกัน**

- หากมีการ “ลักทรัพย์”, “ขโมย”, หรือ “ชิงทรัพย์”:
- แล้วมีคำชัดเจนว่า “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” ปรากฏในข้อความ → ใส่ **[2, 5]**
- ห้ามใส่ [5] หากในข้อความไม่มีคำชัดเจน เช่น “ข่มขู่”, “ทำร้าย”, “ใช้อาวุธ”, “ทำให้เจ็บ”, “ทำให้เลือดออก”, “ยิง”, “แทง”, “ตบ”, “บีบคอ” หรือคำใกล้เคียง
- หากไม่มีการข่มขู่หรือทำร้ายร่างกาย → ใส่เฉพาะ **[2]**

- หากความผิดเกิดจาก “เมาแล้วขับ” หรือ “หลับใน” → ใส่เฉพาะ **[3]** และ **ห้ามใส่ [5]**

- หากมีการ "ฉ้อโกง", "ปลอมเอกสาร" หรือ "หลอกลวง":
- แล้วมีคำชัดเจนว่า “ข่มขู่”, “ทำร้ายร่างกาย”, หรือ “ใช้อาวุธ” ปรากฏในข้อความ → ใส่ **[5, 6]**
- ห้ามใส่ [5] หากในข้

  0%|          | 0/1000 [01:21<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
true_answer = []
for answer in tqdm(result):
    true_answer.append(answer.split('</think>\n\n')[-1])

true_answer[:10]

100%|██████████| 1000/1000 [00:00<00:00, 420861.33it/s]


["[{'จรา': '[5,7]'}, {'เคี้ยน': '[1]'}]",
 "[{'จิดาพัฒน์': '[5]'}]",
 "[{'กรัณยพร': '[3,7]'}]",
 "[{'คำนึงภัช': '[6]'}, {'กฤดาวัติ': '[2]'}]",
 "[{'กอบเกิด': '[3,7]'}]",
 "[{'เจษฎาพันธ์': '[2]'}]",
 "[{'กัณต์พงษ์': '[1]'}]",
 "[{'จันทน์ศรี': '[3,7]'}]",
 "[{'จิรัฐิติใช้': '[5,2]'}]",
 "[{'แก้วเกศ': '[3]'}]"]

In [ ]:
test_df.loc[3:, 'answers'] = true_answer[3:]
test_df.head(10)

,id,story,answers
0,XwlpP5otmI,เคี้ยนแพ้หมากรุกเลยชักปืนขึ้นมายิงจราจนเสียชีวิต,"[{'เคี้ยน': '[5,7]'}, {'จรา': '[1]'}]"
1,1pUXZYyXqQ,จิดาพัฒน์ยิงปืนเล่นน้ำสงกรานต์,[{'จิดาพัฒน์': '[1]'}]
2,drf7jObOL3,กรัณยพรขับรถเมาแล้วชนรถบัสในสถานีขนส่ง ส่งผลให...,"[{'กรัณยพร': '[3,7]'}]"
3,ghD7Mhgjsq,คำนึงภัชทุจริตในการจัดซื้อวัสดุโรงเรียนและปลอม...,"[{'คำนึงภัช': '[6]'}, {'กฤดาวัติ': '[2]'}]"
4,5lfeF2x61n,กอบเกิดขับรถขณะหลับในช่วงกลางวัน ทำให้รถพุ่งข้...,"[{'กอบเกิด': '[3,7]'}]"
5,zgFYbAeF9O,เจษฎาพันธ์ขโมยเงินสดจำนวนมากจากตู้เอทีเอ็มในห้...,[{'เจษฎาพันธ์': '[2]'}]
6,enRAy4uO6C,กัณต์พงษ์อ่านข่าวว่าวันนี้มีพายุรุนแรง,[{'กัณต์พงษ์': '[1]'}]
7,Zklta5lQKO,จันทน์ศรีขับรถชนกลุ่มคนในงานเทศกาลจนมีผู้เสียช...,"[{'จันทน์ศรี': '[3,7]'}]"
8,M2chvwENMI,จิรัฐิติใช้มีดโจมตีในงานเลี้ยงและขโมยของมีค่าใ...,"[{'จิรัฐิติใช้': '[5,2]'}]"
9,akHVq7hmMb,แก้วเกศขับรถเมาแล้วชนกับคนเดินในตรอก ทำให้เกิด...,[{'แก้วเกศ': '[3]'}]


In [ ]:
test_df.to_csv('Qwen3_14B_fine-tuned.csv', index=False)